This is a code exercise for gradient boost using regression tree.
For algorithm, refer to 
https://www.youtube.com/watch?v=3CC4N4z3GJc&t=753s

Part 1: The code block for creating a regression tree and predicting.

In [2]:
# this function choose the best feature given current data set. 
n_seq=8
def BestFeature(Dataset, unvisited_feature,discrete):
    # want to return the best feature and the resulting data split
    # but before this , you can already calculate the the mse of the current data
    orginal_mse=mse_cal(Dataset[y])
    threshold=[]
    mse=[]
    #print(unvisited_feature)
    for feature in unvisited_feature:
        if discrete[feature]=='yes': # this is discrete 
            # feature is name
            # feature level is number?
            # next, for each level in the feature_level, get
            mse.append(SplitDataSet(Dataset,feature, 0.5)[0])
            threshold.append(0.5)
        else: # the feature is continuous 
            # we first generate a threshold sequence 
            thres_seq=[   min(Dataset[feature])+i*(max(Dataset[feature])-min(Dataset[feature]))/(n_seq-1) 
                      for i in range(1,n_seq-1)   ]
            mse_seq= [ SplitDataSet(Dataset,feature, x)[0] for x in thres_seq]
            mse.append(min(mse_seq))
            threshold.append(thres_seq[mse_seq.index(min(mse_seq))])
        
    # we complete the loop with mse  series
    # we now return the best feature and the corresponding threshold 
    # for discrete, the corresponding threshold is always -1 
    best_feature=unvisited_feature[mse.index(min(mse))]
    best_threshold=threshold[mse.index(min(mse))]
    # we also return the splitted dat
    return best_feature,best_threshold
    # best_feature is a string. best_threshold is a value. 

# This function split the data into two parts given the feature and the threshold (value)
def SplitDataSet(Dataset,feature, value):
# we only consider the binary classification case
# given the dataset, feature and value, this function returns splitted data and the resulting MSE!
    Data_left=Dataset[Dataset[feature]<value]
    Data_right=Dataset[Dataset[feature]>=value]
# we want to calculate the mse of the data_left, data_right
    Total_mse=mse_cal(Data_left[y])+mse_cal(Data_right[y])
    return Total_mse,Data_left, Data_right 

import math
from statistics import mean 
def mse_cal(y_data):
    return(sum([pow((i-mean(y_data)),2) for i in y_data]))


def createTree(Dataset, layer,features,discrete):
    # we first need to check whether the layer has already reached the max
    if (layer==maxlayer  ):
        # you have already reached the deepest. all you have to do is to return the average y
        Tree=mean(Dataset[y])
    else:
        best_feature,best_threshold=BestFeature(Dataset, features,discrete)
        Tree = {}  # means that you want to generate a new son tree
        # the new son tree should contain the following features:
        Tree['spInd'] = best_feature # the feature you use to split the data
        featuresNext=features
        featuresNext.remove(best_feature)  # this feature would no longer be used in the next trees
        Tree['spVal'] = best_threshold # the threshold value 
        data_left, data_right = SplitDataSet(Dataset,best_feature, best_threshold)[1],SplitDataSet(Dataset,best_feature, best_threshold)[2]
        Tree['left'] = createTree(data_left,layer+1,featuresNext,discrete)
        Tree['right'] = createTree(data_right,layer+1,featuresNext,discrete)
    return Tree

# the following is for prediction.
def predict(sample, tree):
    if (sample[tree['spInd']]<tree['spVal']):
        pred = predict(sample,tree['left']) if type(tree['left']).__name__=='dict' else tree['left']
    else:
        pred = predict(sample,tree['right']) if type(tree['right']).__name__=='dict' else  tree['right']
    return (pred)

Part 2: The code block for XGBoost 

In [8]:
# XGboost function
def GradientBoost (data , maxlayer, max_tree_number,discrete,learn_rate ):
    n=len(data)
    current_data = data.copy()
    current_predict = [0]*n
    current_tree_series = []
    while (len(current_tree_series)<=max_tree_number):
        # you a whole data set, whose y value is residual. use this to create a tree. : create_tree(current_data, maxlayer, data_features_original)
# thorough this step you get a new tree
        data_features = data_features_original.copy()
        #print(len(current_data))
        #print(discrete)
        new_tree=createTree(current_data, 1, data_features,discrete) # a tree always start from first layer
        
        print(new_tree)
       # add it to tree_series with a learning rate learning rate
       # produce new prediction and get the new residual
        new_predict = [ predict ( current_data.loc[i],new_tree ) for  i in range(0,n) ]
        #print(new_predict)
        current_predict =list(map(lambda x,y:x+y, current_predict , [j * learn_rate for j in new_predict] ))
        current_tree_series.append ( new_tree )
       # print(len(current_predict))
        residual = data[ y ] - current_predict
        # use the residual as the y for the next tree.
        current_data[ y ] = residual
        print(sum(list(map(lambda x: abs(x), residual))))
    return current_tree_series


The next part is a try. we first input the data, and then see whether the XGBoost works.

In [6]:
import numpy as np
import pandas as pd 
data= pd.read_excel('C:/Users/shufe/Dropbox/ds new/pwchkt.xlsx')
data.isnull().any(axis=1) # 判定每行会不会有空值的记录。（某条记录但凡在一个变量上有空值，就认为此记录
data=data.dropna(how='any')
data=data[['age','creditamount','guarantors','housing','foreigner','creditrating']]
data.loc[(data.creditrating == 2),'creditrating']=0
data_features_original = list(data.columns) 
y='creditamount'
data_features_original.remove(y)
# we also need to create a dictionary indicating whether the feature is discrete or not
discrete={'age':'no','creditamount':'no','guarantors':'yes','housing':'yes','foreigner':'yes','creditrating':'yes'}
data_features=data_features_original.copy()



In [7]:
maxlayer = 3
max_tree_number = 15
xixixi = GradientBoost (data , maxlayer, max_tree_number ,discrete,0.25)

{'spInd': 'creditrating', 'spVal': 0.5, 'left': {'spInd': 'age', 'spVal': 50.42857142857143, 'left': 3765.631970260223, 'right': 5434.935483870968}, 'right': {'spInd': 'foreigner', 'spVal': 0.5, 'left': 1932.909090909091, 'right': 3037.5322338830583}}
2783988.18642083
{'spInd': 'creditrating', 'spVal': 0.5, 'left': {'spInd': 'age', 'spVal': 66.14285714285714, 'left': 3299.266063864451, 'right': 8113.426344086022}, 'right': {'spInd': 'foreigner', 'spVal': 0.5, 'left': 1642.9727272727273, 'right': 2581.9023988005997}}
2411874.586160041
{'spInd': 'creditrating', 'spVal': 0.5, 'left': {'spInd': 'foreigner', 'spVal': 0.5, 'left': 6589.265294881298, 'right': 2794.7023439880904}, 'right': {'spInd': 'age', 'spVal': 27.0, 'left': 1796.9733698592797, 'right': 2251.871548737446}}
2161460.3580885925
{'spInd': 'creditrating', 'spVal': 0.5, 'left': {'spInd': 'age', 'spVal': 42.57142857142857, 'left': 2204.2701377501603, 'right': 3275.4296448326913}, 'right': {'spInd': 'foreigner', 'spVal': 0.5, 'lef